<a href="https://colab.research.google.com/github/alexalex6767/Stock-Price-Prediction-with-NLP-from-Yahoo-Finance/blob/main/%E5%85%AC%E5%8F%B8%E8%BC%94%E5%8A%A9%E4%BF%A1%E6%81%AF_%E6%96%B0%E8%81%9E%E9%9D%A2NLP%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps
1. Obtained **"Specific Corporations"** (variable) word data from some **time periods**(variable)，and used Yahoo finance API to obtain the corresponding **stock price**，Characteristic:
    - Time
    - News Source
    - News Title
    - News Content
    - [signal](https://finance.yahoo.com/quote/3105.TWO/chart?p=3105.TWO#eyJpbnRlcnZhbCI6IndlZWsiLCJwZXJpb2RpY2l0eSI6MSwidGltZVVuaXQiOm51bGwsImNhbmRsZVdpZHRoIjoyMi4yNTQ5MDE5NjA3ODQzMTMsImZsaXBwZWQiOmZhbHNlLCJ2b2x1bWVVbmRlcmxheSI6dHJ1ZSwiYWRqIjp0cnVlLCJjcm9zc2hhaXIiOnRydWUsImNoYXJ0VHlwZSI6ImxpbmUiLCJleHRlbmRlZCI6ZmFsc2UsIm1hcmtldFNlc3Npb25zIjp7fSwiYWdncmVnYXRpb25UeXBlIjoib2hsYyIsImNoYXJ0U2NhbGUiOiJsaW5lYXIiLCJzdHVkaWVzIjp7IuKAjHZvbCB1bmRy4oCMIjp7InR5cGUiOiJ2b2wgdW5kciIsImlucHV0cyI6eyJpZCI6IuKAjHZvbCB1bmRy4oCMIiwiZGlzcGxheSI6IuKAjHZvbCB1bmRy4oCMIn0sIm91dHB1dHMiOnsiVXAgVm9sdW1lIjoiIzAwYjA2MSIsIkRvd24gVm9sdW1lIjoiI2ZmMzMzYSJ9LCJwYW5lbCI6ImNoYXJ0IiwicGFyYW1ldGVycyI6eyJ3aWR0aEZhY3RvciI6MC40NSwiY2hhcnROYW1lIjoiY2hhcnQifX19LCJwYW5lbHMiOnsiY2hhcnQiOnsicGVyY2VudCI6MSwiZGlzcGxheSI6IjMxMDUuVFdPIiwiY2hhcnROYW1lIjoiY2hhcnQiLCJpbmRleCI6MCwieUF4aXMiOnsibmFtZSI6ImNoYXJ0IiwicG9zaXRpb24iOm51bGx9LCJ5YXhpc0xIUyI6W10sInlheGlzUkhTIjpbImNoYXJ0Iiwi4oCMdm9sIHVuZHLigIwiXX19LCJsaW5lV2lkdGgiOjIsInN0cmlwZWRCYWNrZ3JvdW5kIjp0cnVlLCJldmVudHMiOnRydWUsImNvbG9yIjoiIzAwODFmMiIsInN0cmlwZWRCYWNrZ3JvdWQiOnRydWUsImV2ZW50TWFwIjp7ImNvcnBvcmF0ZSI6eyJkaXZzIjp0cnVlLCJzcGxpdHMiOnRydWV9LCJzaWdEZXYiOnt9fSwicmFuZ2UiOnsiZHRMZWZ0IjoiMjAyMC0xMi0zMVQxNjowMDowMC40NzJaIiwiZHRSaWdodCI6IjIwMjItMDEtMDJUMTU6NTk6MDAuNDcyWiIsInBlcmlvZGljaXR5Ijp7ImludGVydmFsIjoid2VlayIsInBlcmlvZCI6MX0sInBhZGRpbmciOjB9LCJjdXN0b21SYW5nZSI6eyJzdGFydCI6MTYwOTYwMzIwMDAwMCwiZW5kIjoxNjM5ODQzMjAwMDAwfSwic3ltYm9scyI6W3sic3ltYm9sIjoiMzEwNS5UV08iLCJzeW1ib2xPYmplY3QiOnsic3ltYm9sIjoiMzEwNS5UV08iLCJxdW90ZVR5cGUiOiJFUVVJVFkiLCJleGNoYW5nZVRpbWVab25lIjoiQXNpYS9UYWlwZWkifSwicGVyaW9kaWNpdHkiOjEsImludGVydmFsIjoid2VlayIsInRpbWVVbml0IjpudWxsfV19)
2. Based on the data above to establish **Textual Classification Model(詳情可看論文)** (這邊就是一個完整專案流程)
    - data preprocessing
    - Characteristic Engineering
    - Model Selection and Training
    - Adjust parameter
    - Deploy and Monitor
3. Conclusion

In [ ]:
pip install transformers

In [ ]:
pip install selenium

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import re
import jieba
import tensorflow as tf
from tensorflow import keras
import transformers
import time
import datetime

# 爬蟲相關套件
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver

import warnings
warnings.filterwarnings("ignore")

# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud


plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
%matplotlib inline

In [ ]:
# 1. 取得特定公司的新聞文字資料
# 透過 webdriver 模擬網頁開啟動作，可以模擬真實操作網頁，故可以省下許多AJAX or 其他爬蟲技巧，可以直覺地透過模擬搭配 bs4 萃取需要的網站資訊。


# 參數設定
company_num = '3105'
url = f'https://tw.stock.yahoo.com/quote/{company_num}'


# driver init
driver = webdriver.Chrome('chromedriver.exe')

# get in the url
driver.get(url)   # 進入該網頁
time.sleep(5)     # 等待載入完畢, 若沒有等待可能會造成有些元素還沒有respond

# scroll down
js="var q=document.documentElement.scrollTop=10000"
driver.execute_script(js)
time.sleep(3)

# 透過BeautifulSoup解析網頁內容
soup = BeautifulSoup(driver.page_source, "html.parser")

In [ ]:
# 這邊有抓到一些不適用的東西, 可以透過 "穩懋" 判斷文章內容，沒有就剔除

for e in soup.find_all(class_='js-stream-content Pos(r)'):
    print(e.find_all('a')[0].get('href'))
    print('\n\n')

In [ ]:
def get_company_news(company_num):
    """
        透過 selenium 網頁操作模擬套件去模擬人工操作，因為 Yahoo Stock 網站有使用非同步更新頁面的方法，
        透過 selenium 可以大幅度減低抓取難度，另外，因為新聞內容不需要太頻繁更新(對比即時股價)，
        故 selenium 的速度限制將不會是一個問題。
    """
    url = f'https://tw.stock.yahoo.com/quote/{company_num}'
    news_urls = []
    options = webdriver.ChromeOptions()
    options.add_argument(' --headless')   # 背景執行，如果沒有使用，需要把該網頁開始，不然js執行會出現問題
    driver = webdriver.Chrome('chromedriver.exe', options=options)  # 預設將 chromedriver.exe 放同一 level 
    driver.get(url)   # 進入該網頁
    time.sleep(5)     # 等待載入完畢, 若沒有等待可能會造成有些元素還沒有respond
    
    # scroll down
    for i in range(10):
        print('下拉等待...')
        js="window.scrollTo(0,document.body.scrollHeight)"
        driver.execute_script(js)
        time.sleep(3)
        print('下拉完成')
    
    # 透過BeautifulSoup解析網頁內容
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    for e in soup.find_all(class_='js-stream-content Pos(r)'):
        try:
            news_url = e.find_all('a')[0].get('href')
            news_source = e.find(class_='C(#959595) Fz(13px) C($c-secondary-text)! D(ib) Mb(6px)').text
            news_source = news_source[:news_source.index('•')]
            news_title = e.find(class_='Mt(0) Mb(8px)').text
            print(news_title, news_source)
            news_urls.append((news_url, news_source))
        except Exception as e:
            print(e)
    return news_urls


company_num = '3105'
news_urls = get_company_news(company_num)

In [ ]:
# 單一文章看看可否直接抓出內容，或許搭配 threading 快速下載
def is_in_timerange(t, datetime_start, datetime_end):
    if datetime_start <= t <= datetime_end:
        return True
    return False


def get_news(url):
    """
        透過 requests 套件取得網頁內容，再搭配 Beautifulsoup 模組去解析網頁內容，
        取得需要的網頁內容: 新聞標題、新聞來源、新聞發布時間、新聞內文。
    """
    # 取得網頁內容
    resp = requests.get(url)
    soup_news = BeautifulSoup(resp.content, 'html.parser')

    # 解析並取得需要的特徵
    # 這些資料都需要清理一下
    title = soup_news.find(class_='caas-header').text
    source = soup_news.find(class_='caas-author-byline-collapse').text
    time_text = soup_news.find(class_='caas-attr-time-style').text   # 還需要處理
    news_content = soup_news.find(class_='caas-body').text
    
    print(title, source, time_text)
    return title, source, time_text, news_content
    
    
# url = 'https://tw.stock.yahoo.com/news/%E5%85%AC%E5%91%8A-%E7%A9%A9%E6%87%8B%E4%BB%A3%E9%87%8D%E8%A6%81%E5%AD%90%E5%85%AC%E5%8F%B8chainwin-biotech-agrotech-cayman-104429660.html'
# get_news(url)
rows = []

for e in news_urls:
    row = list(get_news(e[0]))
    row[1] = e[1]
    rows.append(row)

df = pd.DataFrame(rows, columns=['title', 'source', 'time', 'news_content'])
df

In [ ]:
# test regex

r = "（中央社記者蔡芃敏台北2021年12月22日電）光電科技工業協進會（PIDA）今天指出，全球..."
reg = re.compile(r'[()（），]')
# reg = re.compile('\\.+?(?=\B|$【（：）】，。)')
re.sub(reg, string = r, repl = '')

In [ ]:
def process_datetime(time_text):
    """
        範例: '2021年6月10日 上午11:51·1 分鐘 (閱讀時間)'
    """
    is_afternoon = False
    if '上午' in time_text:
        is_afternoon = True
        time_text = time_text.replace('上午', '')
    else:
        time_text = time_text.replace('下午', '')
    
    if '·' in time_text:
        time_text = time_text[:time_text.index('·')]
    if is_afternoon:
        return datetime.datetime.strptime(time_text, "%Y年%m月%d日 %H:%M")
    else:
        return datetime.datetime.strptime(time_text, "%Y年%m月%d日 %H:%M") + datetime.timedelta(hours=8)
    
    
def clean(text):
    # 移雛一些特殊符號以及數字，這些透過觀察 raw data 而知
    reg = re.compile('[\\d\-)(?=|$【（：）】《，》。]')   
    text = text.apply(lambda r: re.sub(reg, string = r, repl = ''))
    # 移除 hashtag 的符號'#'
    text = text.apply(lambda r: r.replace('#', ''))
    # 移除人名標記
    reg = re.compile('@\w+')
    text = text.apply(lambda r: re.sub(reg, string = r, repl = '@'))
    # 移除網址
    reg = re.compile('https?\S+(?=\s|$)')
    text = text.apply(lambda r: re.sub(reg, string = r, repl = 'www'))
    # 全部小寫化
    text = text.apply(lambda r: r.upper())
    text = text.apply(lambda r: r.replace(' ', ''))
    
    return text

def process_text(df):
    """
        將文字資料根據實際狀況前處理資料。
        
        步驟:
            1. 將 Stop words 移除。
            2. 將不應納入考量的資訊移除，如數字、標點符號、特殊符號(已做完)
            3. 將一些 domain-know-how 的 word 加入字典(分詞會優先參考): 如行業術語，此處參考 **中文財務情緒字典** 
            4. 分詞
    """
    # 1.
    import jieba.analyse
    jieba.analyse.set_stop_words('./datasets/停用詞-繁體中文.txt')
    
    # 3. 這部分先沒有做，但可以當作擴充。
    
    # 4.
    # jieba.load_userdict('./datasets/new_dict.txt')
    df['cut'] = df['cleaned_content'].apply(lambda sentence: ' '.join(list(jieba.cut(sentence))))
    
    return df
    
    

# process_datetime('2021年6月10日 上午11:51·1 分鐘 (閱讀時間)')
df['cleaned_time'] = df['time'].apply(func=lambda x : process_datetime(x))
df['datetime_str'] = df['cleaned_time'].apply(func=lambda x : str(x+datetime.timedelta(days=1))[:10])
df['cleaned_content'] = clean(df['news_content'])
df['cleaned_title'] = clean(df['title'])
df

In [ ]:
# 處理股價資料
# 建立 交易信號: (收盤-開盤) / 收盤 * 100

def transform_price_to_label(price_trend, threshold):
    """
        將價格趨勢轉換成 label，離散化，這是一個 Reframing 的技巧。
    """
    if price_trend >= threshold:
        return 1
    elif price_trend < -threshold:
        return 0
    return -1

stocks = pd.read_csv('3105.TWO.csv')
stocks['price_trend'] = (stocks['Close'] - stocks['Open']) / stocks['Close'] * 100

threshold = 1
stocks['label'] = stocks['price_trend'].apply(func=lambda x: transform_price_to_label(x, threshold))
stocks

In [ ]:
# 合併

stocks['clean_time'] = stocks['Date'].apply(func=lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
stocks

In [ ]:
stocks[stocks['Date'] == '2021-01-05'].iloc[0, -2]

In [ ]:
def get_label(df):
    """
        透過時間判斷搭配stocks資料得取得 label。
    """
    mapping = {}
    for datetime_str in stocks['Date']:
        #print(datetime_str)
        mapping[datetime_str] = stocks[stocks['Date'] == datetime_str].iloc[0, -2]
    
    df['label'] = df['datetime_str'].apply(func=lambda x: mapping[x] if x in mapping else 0)
    
get_label(df)
df

In [ ]:
# 圖示化 標籤分布

df_bar = pd.DataFrame({'label': ['漲幅訊號', '平穩訊號', '跌幅訊號'], 'count': [(stocks['label'] == 1).sum(), (stocks['label'] == 0).sum(), (stocks['label'] == -1).sum()]})

df_bar.plot.bar(x='label', y='count', rot=0)

In [ ]:
# 斷詞

df = process_text(df)
df.head()

In [ ]:
# 向量化: tf-idf

# corpus = df.ilo
corpus = df.loc[:, 'cut'].values
vectorizer = TfidfVectorizer(max_features=3000)
x_tf_idf = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names()
print(x_tf_idf.shape)

In [ ]:
# 訓練資料

X = pd.DataFrame(x_tf_idf.toarray(), columns=vectorizer.get_feature_names())
X

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, auc
from sklearn.model_selection import cross_val_score

models = {
    'rf': RandomForestClassifier(),
    'gdbt': GradientBoostingClassifier(),
    'lr': LogisticRegression(),
    'svm': SVC(),
    'nb': MultinomialNB()
}
models_result = {}

for name, model in models.items():
    print(name)
    scores = cross_val_score(estimator = model, X=X, y=df['label'], cv=5)
    print(scores)
    print(f'平均acc: {np.mean(scores)}')

> 基本上，因為本身資料少又有缺陷，對於建立一個基於此的模型，效果自然不會太好。 之後用預訓練模型可能會有更好的效果。

In [ ]:
# wordcloud based on tf-idf

# corpus = ['Hi what are you accepting here do you accept me',
# 'What are you thinking about getting today',
# 'Give me your password to get accepted into this school',
# 'The man went to the tree to get his sword back',
# 'go away to a far away place in a foreign land']

# vectorizer = TfidfVectorizer(stop_words='english')
# vecs = vectorizer.fit_transform(corpus)
# feature_names = vectorizer.get_feature_names()


dense = x_tf_idf.todense()
lst1 = dense.tolist()
df_cloud = pd.DataFrame(lst1, columns=vectorizer.get_feature_names())
# df.T.sum(axis=1)

cloud = WordCloud(background_color="white", max_words=200, font_path = "C:\Windows\Fonts\kaiu").generate_from_frequencies(df_cloud.T.sum(axis=1))
plt.imshow(cloud.to_file('文字雲.png'))
plt.axis("off")
plt.show()

In [ ]:
# 詞頻版本

import wordcloud

def draw_wordcloud(corpus):
    """
        functionality:
            draw the wordcloud
        Args:
            corpus: str
    """
    words = list(jieba.cut(corpus, cut_all = False))   # 分詞
    #words = [e for e in words if e.isalpha()]        # 
    words = [e for e in words if len(e) != 1]        # 只要長度是2以上的word
    words = " ".join(words)
    
    cloud = wordcloud.WordCloud(background_color = "black",
                                font_path = "C:\Windows\Fonts\kaiu",   # 字型路徑
                                scale = 2,                             # 調整解析度
                                width = 1000,                          # 圖片長度
                                height = 600,                          # 圖片寬度
                                min_font_size = 20,
                                max_words = 200)
    cloud = cloud.generate(words)                    # 產生文字雲
    plt.figure(figsize = (15, 15))
    plt.axis("off")
    plt.imshow(cloud)
    plt.show()

    
def get_class_wordcloud(df):
    for i in df['label'].unique():
        print(i)
        corpus = []
        temp_df = df[(df['label'] == i)]
        for j in range(len(temp_df)):
            corpus.append(temp_df.iloc[j, -1])
        corpus = ''.join(corpus)
        draw_wordcloud(corpus)
        
        
get_class_wordcloud(df)

In [ ]:
import lda
from sklearn.feature_extraction.text import CountVectorizer 

vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names()   # 所有的特征词，即关键词
print (word)    


# 訓練模型
weight = X_bow.toarray()
model = lda.LDA(n_topics = 3, n_iter = 500, random_state = 1)
model.fit(np.asarray(weight))
# 主题-詞 分布
topic_word = model.topic_word_  # 生成主题以及主题中词的分布
print("topic-word:\n", topic_word)

In [ ]:
# 计算topN关键词
n = 5    
for i, word_weight in enumerate(topic_word):  
    print("word_weight:\n", word_weight)
    distIndexArr = np.argsort(word_weight)
    print("distIndexArr:\n", distIndexArr)
    topN_index = distIndexArr[:-(n+1):-1]
    print("topN_index:\n", topN_index) # 权重最在的n个
    topN_words = np.array(word)[topN_index]    
    print(u'*Topic {}\n- {}'.format(i, ' '.join(topN_words))) 
# 绘制主题-词分布图
import matplotlib.pyplot as plt  
f, ax= plt.subplots(2, 1, figsize=(6, 6), sharex=True)  
for i, k in enumerate([0, 1]):         #两个主题
    ax[i].stem(topic_word[k,:], linefmt='b-',  
               markerfmt='bo', basefmt='w-')  
    ax[i].set_xlim(-2,20)  
    ax[i].set_ylim(0, 1)  
    ax[i].set_ylabel("Prob")  
    ax[i].set_title("topic {}".format(k))  
ax[1].set_xlabel("word")  
plt.tight_layout()  
plt.show()

In [ ]:
# 文档-主题分布  
doc_topic = model.doc_topic_ 
print("type(doc_topic): {}".format(type(doc_topic)))  
print("shape: {}".format(doc_topic.shape)) 
label = []        
for i in range(10):  
    print(doc_topic[i])
    topic_most_pr = doc_topic[i].argmax()  
    label.append(topic_most_pr)  
    print("doc: {} topic: {}".format(i, topic_most_pr))  
print(label)    # 前10篇文章对应的主题列表

In [ ]:
# 绘制文档-主题分布图  
import matplotlib.pyplot as plt    
f, ax= plt.subplots(6, 1, figsize=(8, 8), sharex=True)    
for i, k in enumerate([0,1,2,3,8,9]):    
    ax[i].stem(doc_topic[k,:], linefmt='r-',    
               markerfmt='ro', basefmt='w-')    
    ax[i].set_xlim(-1, 2)     #x坐标下标  
    ax[i].set_ylim(0, 1.2)    #y坐标下标  
    ax[i].set_ylabel("Probability")    
    ax[i].set_title("Document {}".format(k))    
ax[5].set_xlabel("Topic")  
plt.tight_layout()  
plt.show() 